# Converting Images to a Cloud Volume Image Layer

This module demonstrates how to convert a sequence of image files into a CLoudVolume image layer. The images can be in any standard format, including TIFF, PNG, JPEG, BMP, and many more. See the [scikit-image docs](https://scikit-image.org/docs/dev/api/skimage.io.html "skimage.io Documentation") for more information about file types.

In [1]:
# We need to add balsam and the modules it depends on to the Python search paths. 
import sys
sys.path.insert(0,'/soft/datascience/Balsam/0.3.5.1/env/lib/python3.6/site-packages/')
sys.path.insert(0,'/soft/datascience/Balsam/0.3.5.1/')

# We also need balsam and postgresql to be in the path. (Misha suggests this may not be necessary)
import os
os.environ['PATH'] ='/soft/datascience/Balsam/0.3.5.1/env/bin/:' + os.environ['PATH']
os.environ['PATH'] +=':/soft/datascience/PostgreSQL/9.6.12/bin/'
try:
    import balsam
except ImportError:
    print('Cannot find balsam, make sure balsam is installed or it is available in Python search paths')
    
os.environ["BALSAM_DB_PATH"]='/lus/theta-fs0/projects/connectomics_aesp/balsam_database/'

from balsam_helper import *

# Import widgets
from ipywidgets import interact, interactive
from ipywidgets import fixed, interact_manual 
from ipywidgets import Textarea, widgets, Layout, Accordion
from ipywidgets import VBox, HBox, Box, Text, BoundedIntText

env_preamble = '/lus/theta-fs0/projects/connectomics_aesp/software/HappyNeuron/macros_theta/theta_build_preamble.sh'

## Set up the Application

Before submitting image conversion jobs, make sure there is a Balsam application available to you. Here, we set up a HappyNeuron application and img2cv workflow. The application will run the img2cv executable installed with HappyNeuron.

In [2]:
add_app(
    'HappyNeuron_img2cv',
    'python /lus/theta-fs0/projects/connectomics_aesp/software/HappyNeuron/happyeuron/io/img_to_cloudvolume.py',  # 'python img_to_cloudvolume',
    description='Convert images to a CloudVolume layer.',
    envscript=env_preamble
)

OperationalError: could not connect to server: Connection refused
	Is the server running on host "thetalogin5" (10.236.1.194) and accepting
	TCP/IP connections on port 54551?


## Set up the Workflow and Job

This is where job parameters will be set and added to a workflow in the application.

In [ ]:
img_path = '/lus/theta-fs0/projects/connectomics_aesp/pipeline_data/img2cv/images'
cv_path = '/lus/theta-fs0/projects/connectomics_aesp/pipeline_data/img2cv/precomputed/image'
ext = '.png'

args = f'--input {img_path} --output {cv_path} --ext {ext}'

add_job(
    'convert_srb',  # Job Name
    'img2cv_srb',   # Workflow Name
    'HappyNeuron_img2cv'',
    description='Convert the SRB stack to a CloudVolume image layer.',
    args=args,
    num_nodes=8,
    ranks_per_node=4
)

## Run the Conversion

In [ ]:
submit(project='connectomics_aesp',
   queue='debug-flat-quad',
   nodes=8,
   wall_minutes=20,
   wf_filter='img2cv_srb'
)